**Table of contents**<a id='toc0_'></a>    
- [AoC 2024-07](#toc1_)    
  - [Part 1](#toc1_1_)    
  - [Second part: 3 operations +,*,||](#toc1_2_)    
  - [Optimization 1: Reverse travelling](#toc1_3_)    
  - [Optmization 2: Ternary Tree DFS](#toc1_4_)    
  - [Optimization 1 and 2: Reverse with DFS](#toc1_5_)    
  - [Final: Iteration with Stack instead of Recursion](#toc1_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[AoC 2024-07](#toc0_)

## <a id='toc1_1_'></a>[Part 1](#toc0_)

In [1]:
def parse_input(fname):
    with open(fname, "r") as f:
        lines = f.readlines()
    output = []
    for line in lines:
        items = line.split(":")
        expected = int(items[0])
        terms = [int(x) for x in items[1].split()]
        output.append((expected, terms))
    return output

In [2]:
from typing import Literal

Operation = Literal[0, 1]


def check_operation(expected, terms, operations):
    assert len(terms) == len(operations) + 1
    total = terms[0]
    for i in range(len(operations)):
        # zero +, one *
        if operations[i] == 0:
            total += terms[i + 1]
        else:
            total *= terms[i + 1]
        if total > expected:
            return False
    return total == expected


# define Op class which can be either 0 or 1
def int_to_operations(i: int, n_ops: int) -> list[Operation]:
    binary = bin(i)[2:].zfill(n_ops)
    operations = [int(x) for x in binary]
    return operations


def check_terms(expected, terms):
    n_ops = len(terms) - 1
    # generate all combinatios of + and * of len n_ops in binary numbers
    for i in range(2**n_ops):
        operations = int_to_operations(i, n_ops)
        if check_operation(expected, terms, operations):
            return True
    return False


In [ ]:
output = parse_input("input.txt")
total = 0
for expected, reversed_terms in output:
    can_be_true = check_terms(expected, reversed_terms)
    if can_be_true:
        total += expected

print(total)

2941973819040


## <a id='toc1_2_'></a>[Second part: 3 operations +,*,||](#toc0_)

In [ ]:
from itertools import product
from typing import Iterable, Literal

Operation = Literal[0, 1, 2]


def iterable_operators(n_ops: int, unique_ops: int = 3) -> Iterable[tuple[Operation]]:
    possible_values = list(range(unique_ops))
    return product(possible_values, repeat=n_ops)


def check_operation_extended(
    expected: int, terms: list[int], operations: list[Operation]
) -> bool:
    assert len(terms) == len(operations) + 1
    total = terms[0]
    for i in range(len(operations)):
        # zero +, one *, two concatenation
        if operations[i] == 0:
            total += terms[i + 1]
        elif operations[i] == 1:
            total *= terms[i + 1]
        elif operations[i] == 2:
            # concatenation
            total = int(str(total) + str(terms[i + 1]))
        if total > expected:
            return False
    return total == expected


def check_terms_extended(expected, terms):
    n_ops = len(terms) - 1
    possible_operations = iterable_operators(n_ops)
    # generate all combinatios of + and * of len n_ops in binary numbers
    for operations in possible_operations:
        if check_operation_extended(expected, terms, operations):
            return True
    return False


output = parse_input("input.txt")
total = 0
for expected, reversed_terms in output:
    can_be_true = check_terms_extended(expected, reversed_terms)
    if can_be_true:
        total += expected

print(total)
total_correct = total

249943041417600


## <a id='toc1_3_'></a>[Optimization 1: Reverse travelling](#toc0_)

In [ ]:
from typing import Iterable, Literal

Operation = Literal[0, 1, 2]


def iterable_operators(n_ops: int, unique_ops: int = 3) -> Iterable[tuple[Operation]]:
    possible_values = list(range(unique_ops))
    return product(possible_values, repeat=n_ops)


def check_operation_extended(
    expected: int, terms: list[int], operations: list[Operation]
) -> bool:
    assert len(terms) == len(operations) + 1
    total = expected
    for term, op in zip(reversed(terms), reversed(operations), strict=False):
        # zero +, one *, two concatenation
        if op == 0:
            # reverse sum
            total -= term
            if total < terms[0]:
                return False
        elif op == 1:
            # reverse multiplication
            total /= term
            if total.is_integer():
                total = int(total)
            else:
                return False
        elif op == 2:
            # reverse concatenation
            total_str = str(total)
            if total_str.endswith(str(term)):
                total_str = total_str[: -len(str(term))]
                if len(total_str) == 0:
                    return False
                total = int(total_str)
            else:
                return False
    return total == terms[0]


def check_terms_extended(expected, terms):
    n_ops = len(terms) - 1
    possible_operations = iterable_operators(n_ops)
    # generate all combinatios of + and * of len n_ops in binary numbers
    for operations in possible_operations:
        if check_operation_extended(expected, terms, operations):
            return True
    return False


output = parse_input("input.txt")
total = 0
for expected, reversed_terms in output:
    can_be_true = check_terms_extended(expected, reversed_terms)
    if can_be_true:
        total += expected

print(total)
assert total == total_correct

249943041417600


## <a id='toc1_4_'></a>[Optmization 2: Ternary Tree DFS](#toc0_)

In [ ]:
def next_node_recursive(n, terms, expected):
    # prune if n is already larger than expected
    if n > expected:
        return False
    if len(terms) == 0:
        return n == expected
    term = terms[0]
    # operations = [0, 1, 2]
    return (
        next_node_recursive(n + term, terms[1:], expected)
        or next_node_recursive(n * term, terms[1:], expected)
        or next_node_recursive(int(str(n) + str(term)), terms[1:], expected)
    )


def check_operation_recursive(expected: int, terms: list[int]) -> bool:
    is_good = next_node_recursive(terms[0], terms[1:], expected)
    return is_good


def check_terms_extended(expected, terms):
    if check_operation_recursive(expected, terms):
        return True
    return False


output = parse_input("input.txt")
total = 0
for expected, reversed_terms in output:
    can_be_true = check_terms_extended(expected, reversed_terms)
    if can_be_true:
        total += expected

print(total)
assert total == total_correct

249943041417600


## <a id='toc1_5_'></a>[Optimization 1 and 2: Reverse with DFS](#toc0_)

In [13]:
%%timeit


def check_validity_reverse_recursive(
    remaining: int, reversed_terms: list[int], final_remaining: int
) -> bool:
    """Checks validity of the terms

    It applies two optimizations:
    1. DFS with pruning if it is already impossible to reach the expected value
    2. Reverse the application of the operations, to detect impossible branches earlier:
    - if the expected is not multiple of the last term, * is impossible
    - if the expected is smaller than the last term, + is impossible
    - if the expected does not end with the last term, concatenation is impossible

    Parameters
    ----------
    remaining : int
        Current value obtained.
    reversed_terms : list[int]
        Remaining terms to be applied, in reverse order.
    final_remaining : int
        Target value to achieve.

    Returns
    -------
    bool
        True if the target value is achievable, False otherwise.
    """
    # Prune invalid paths
    if remaining < final_remaining or not remaining.is_integer():
        return False
    remaining = int(remaining)

    # Base case: check if target is reached
    if len(reversed_terms) == 0:
        return remaining == final_remaining

    term = reversed_terms[0]

    # Attempt reverse concatenation
    remaining_str = str(remaining)
    if remaining_str.endswith(str(term)):
        # need that the remaining string is not empty
        remaining_concat = (
            int(remaining_str[: -len(str(term))])
            if len(remaining_str) > len(str(term))
            else None
        )
        if remaining_concat and check_validity_reverse_recursive(
            remaining_concat, reversed_terms[1:], final_remaining
        ):
            return True

    # Attempt reverse addition and multiplication
    if check_validity_reverse_recursive(
        remaining - term, reversed_terms[1:], final_remaining
    ):
        return True
    if check_validity_reverse_recursive(
        remaining / term, reversed_terms[1:], final_remaining
    ):
        return True

    # No valid path found
    return False


def check_operation_reverse_recursive(expected: int, terms: list[int]) -> bool:
    # reversed terminology
    remaining = expected
    last_remaining = terms[0]
    reversed_terms = list(reversed(terms[1:]))

    is_good = check_validity_reverse_recursive(
        remaining, reversed_terms, last_remaining
    )
    return is_good


output = parse_input("input.txt")
total = 0
for expected, terms in output:
    can_be_true = check_operation_reverse_recursive(expected, terms)
    if can_be_true:
        total += expected

assert total == total_correct

8.8 ms ± 102 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
def check_validity_reverse_recursive_memory(
    remaining: int,
    reversed_terms: list[int],
    final_remaining: int,
    operations: list[Operation],
) -> tuple[bool, None | list[Operation]]:
    """Checks validity of the terms saving the operations

    Parameters
    ----------
    remaining : int
        current value obtained
    reversed_terms : list[int]
        remaining terms to be applied, reversed
    final_remaining : int
        expected value to obtain at the end

    Returns
    -------
    bool
        True if it is possible to obtain the expected value, False otherwise
    """
    # prune if remaining is < final_remaining
    if remaining < final_remaining:
        return False, None
    # prune if remaining is not integer (after division)
    if remaining.is_integer():
        remaining = int(remaining)
    else:
        return False, None
    # condition to check if it is valid and end of the recursion
    if len(reversed_terms) == 0:
        if remaining == final_remaining:
            return True, operations[::-1]
        else:
            return False, None

    term = reversed_terms[0]
    # reverse concatenation
    is_cocat_possible = True
    remaining_str = str(remaining)
    if remaining_str.endswith(str(term)):
        remaining_str = remaining_str[: -len(str(term))]
        if len(remaining_str) == 0:
            is_cocat_possible = False
        else:
            remaining_concat = int(remaining_str)
    else:
        is_cocat_possible = False
    # prune this branch if cond2 is False
    if is_cocat_possible:
        cond2, operations2 = check_validity_reverse_recursive_memory(
            remaining_concat, reversed_terms[1:], final_remaining, operations + ["|"]
        )
    else:
        cond2 = False

    if cond2:
        return True, operations2

    cond0, operations0 = check_validity_reverse_recursive_memory(
        remaining - term, reversed_terms[1:], final_remaining, operations + ["+"]
    )
    if cond0:
        return True, operations0
    cond1, operations1 = check_validity_reverse_recursive_memory(
        remaining / term, reversed_terms[1:], final_remaining, operations + ["*"]
    )
    if cond1:
        return True, operations1

    return False, None

## <a id='toc1_6_'></a>[Final: Iteration with Stack instead of Recursion](#toc0_)

In [53]:
def check_validity_reverse_iterative(expected: int, terms: list[int]) -> bool:
    """
    Checks if the reversed terms can yield the expected final_remaining using reverse operations.

    Parameters
    ----------
    expected : int
        Value to achieve with all the terms.
    terms : list[int]
        Terms to be applied.

    Returns
    -------
    bool
        True if the target value is achievable, False otherwise.
    """
    first_term = terms[0]
    reversed_terms = terms[1:][::-1]

    # list is perfect for LIFO stack
    stack = [(expected, reversed_terms)]
    while stack:
        current, terms = stack.pop()

        # Base case: check if target is reached after all operations
        if not terms:
            if current == first_term:
                return True
            continue

        term = terms[0]
        next_terms = terms[1:]

        # Attempt reverse concatenation
        current_str = str(current)
        if current_str.endswith(str(term)):
            concatenated_part = current_str[: -len(str(term))]
            if concatenated_part and int(concatenated_part) >= first_term:
                stack.append((int(concatenated_part), next_terms))

        # Attempt reverse addition and multiplication
        if current - term >= first_term:
            stack.append((current - term, next_terms))
        div = current / term
        if div.is_integer() and div >= first_term:
            stack.append((int(div), next_terms))

    return False


output = parse_input("input.txt")

In [62]:
%%timeit

total = sum(
    expected
    for expected, terms in output
    if check_validity_reverse_iterative(expected, terms)
)

assert total == total_correct

4.73 ms ± 60.2 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
